In [1]:
import os
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction/research'

In [2]:
os.chdir("../")
%pwd

'/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_file: Path
    model_1: Path
    model_1_scaler: Path
    model_2: Path
    model_params: dict

In [17]:
from churn_pred.constants import *
from churn_pred.utils.main_utils import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.model_params
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir=Path(config.root_dir),
            train_file=Path(config.train_file),
            model_1=Path(config.model_1),
            model_1_scaler=Path(config.model_1_scaler),
            model_2=Path(config.model_2),
            model_params=params
        )
        
        return model_trainer_config

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import pandas as pd
import pickle
import numpy as np

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def data_prepatarion(self):
        train_data = pd.read_csv(self.config.train_file)
        train_data = train_data.replace([np.inf, -np.inf], np.nan)
        
        lg_y_train = train_data['churn_next_month']
        xgb_y_train = train_data['early_churn']
        
        X_train = train_data.drop(columns=['churn_next_month', 'early_churn'])
        
        return X_train, lg_y_train, xgb_y_train
    
    def logistic_regression_trainer(self, X_train, y_train):
        params = self.config.model_params.logistic_regression.to_dict()
        model = LogisticRegression(**params)
        scaler = StandardScaler()
        
        pipe = Pipeline([
            ('scaler', scaler),
            ('clf', model)
        ])
        
        pipe.fit(X_train, y_train)
        
        with open(self.config.model_1,'wb') as f:
            pickle.dump(model, f)
        
        with open(self.config.model_1_scaler, 'wb') as f:
            pickle.dump(scaler, f)
    
    def xgb_classifier_trainer(self, X_train, y_train):
        params = self.config.model_params.xgb_classifier.to_dict()
        model = XGBClassifier()
        model.set_params(**params)
        
        model.fit(X_train, y_train)
        
        with open(self.config.model_2,'wb') as f:
            pickle.dump(model, f)
    
    def train(self):
        X_train, lg_y_train, xgb_y_train = self.data_prepatarion()
        
        self.logistic_regression_trainer(X_train, lg_y_train)
        
        self.xgb_classifier_trainer(X_train, xgb_y_train)

In [19]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise e

[2025-06-24 14:45:30,344: INFO: main_utils: yaml file: config/config.yaml loaded successfully]
[2025-06-24 14:45:30,350: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2025-06-24 14:45:30,354: INFO: main_utils: yaml file: schema.yaml loaded successfully]
[2025-06-24 14:45:30,356: INFO: main_utils: created directory at: artifacts]
[2025-06-24 14:45:30,358: INFO: main_utils: created directory at: artifacts/model_trainer]


/mnt/cb03386d-9344-47b1-82f9-868fbb64b4ae/python_projects/Churn_Customer_Lifetime_Value_prediction/env/lib/python3.10/site-packages/xgboost/training.py:183: UserWarning: [14:45:31] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "silent" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
